In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.utils.data as data
from torchvision import transforms




In [ ]:
class YOLODataset(data.Dataset):

    def __init__(self, image_dir, label_dir, num_classes, image_size):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.num_classes = num_classes
        self.image_size = image_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.data = self.load_dataset()

    def load_dataset(self):
        dataset = []
        for image_file in os.listdir(self.image_dir):
             if image_file.lower().endswith((".jpg", ".jpeg")):
                image_path = os.path.join(self.image_dir, image_file)
                label_path = os.path.join(self.label_dir, image_file.replace(".jpg", ".txt").replace(".jpeg", ".txt"))

                image = cv2.imread(image_path)
                image = cv2.resize(image, (self.image_size, self.image_size))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    labels = []
                    for line in lines:
                        class_index, center_x, center_y, width, height = map(float, line.strip().split())
                        labels.append([class_index, center_x, center_y, width, height])

                dataset.append({'image': image, 'labels': labels})
        return dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index]['image']
        labels = self.data[index]['labels']

        image = self.transform(image)
        target = torch.zeros((len(labels), 5))

        for i, label in enumerate(labels):
            class_index, center_x, center_y, width, height = label
            target[i, 0] = class_index
            target[i, 1] = center_x
            target[i, 2] = center_y
            target[i, 3] = width
            target[i, 4] = height
        max_objects = 50  # Adjust this based on your needs
        if len(labels) < max_objects:
            pad_size = max_objects - len(labels)
            padding = torch.zeros((pad_size, 5))
            target = torch.cat((target, padding), dim=0)
        return image, target



In [ ]:
# Define dataset and model parameters
image_dir = '/content/drive/MyDrive/Personal Info/Dataset/Skin burn/images'
label_dir = '/content/drive/MyDrive/Personal Info/Dataset/Skin burn/labels'
num_classes = 3
image_size = 224



In [ ]:
# Create YOLO dataset instance
dataset = YOLODataset(image_dir, label_dir, num_classes, image_size)

# Create data loader for training
batch_size = 16
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Iterate through data loader
for i, j in train_loader:
    print(j)

tensor([[[2.0000, 0.4005, 0.4519, 0.5753, 0.8444],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[1.0000, 0.5717, 0.5138, 0.2975, 0.4309],
         [1.0000, 0.5860, 0.1519, 0.1541, 0.1823],
         [1.0000, 0.8763, 0.2044, 0.2115, 0.2762],
         ...,
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[2.0000, 0.3691, 0.6120, 0.1636, 0.1749],
         [2.0000, 0.4691, 0.8005, 0.1164, 0.1475],
         [2.0000, 0.6636, 0.2213, 0.1273, 0.1585],
         ...,
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        ...,

        [[1.0000, 0.

In [ ]:
len(dataset)

300